# 코인마켓

## 프로젝트 개요

1. 많은 사람들이 해외여행 후 남은 외화 동전을 그대로 가져오게 된다.

2. 동전은 한국에서 처리할 수 없기 때문에, 그 나라로 보내게 되는데 이때 지폐보다 무거운 동전은 항공 비용이 발생하게 됨.<br/>⟶ 환율이 **50%**로 적용

3. 본 서비스는 환율의 **70%**로 거래하며, 유저끼리 외화 동전을 거래하게 되는데 **구매자는 은행보다 30%** 저렴하고 **판매자는 20%** 비싸게 팔 수 있으므로 모든 고객에서 이윤이 발생한다.

4. 또한, AI를 이용하여 외화 동전을 한번에 분류하고 환율을 적용 시킬 수 있다면, 일일이 검색하는 수고로움을 덜 수 있을 것이다.

## 프로젝트 목표

###1. 간단한 외화 동전 거래 플랫폼 제작

###2. 외화를 Detection하고 Classification 할 수 있는 AI 모델 제작

## 프로젝트 Blackpoint

- 돈과 관련된 결제 기능이 존재하므로, 데이터의 무결성을 확인하는 작업이 필요함

- Business Model을 생각한다면 거래 최소 금액을 정해놓고 택배 방식(?)

- GPU가 없는 VM이고, AI 모델의 적정 Class는 몇개가 필요할 것인가?

## 1. OneStage or TwoStage(?)

- OneStage Process: 빠르고 거의 **실시간**으로 데이터 처리 가능

- TwoStage Process: 좀 더 **정확**하며, 좀 더 많은 Class 훈련 가능

###1.1. Hough Transform을 이용한 객체 인식 (폐기)

- https://blog.naver.com/samsjang/220588392347 : Hough Transform에 대한 간략한 이론

- 비슷한 픽셀 크기의 Image 및 비슷한 크기의 객체에 대해서는 인식률이 좋다

- 본 서비스에서는 Client가 외화 Image를 촬영하게 되므로 인식률이 매번 좋다고 장담할 수 없다

[본 그림에 하이퍼 파라미터 튜닝 후](https://kdt-gitlab.elice.io/ai_track/class05/ai_project/team07/7zip/uploads/b38de71d09122ad1cb0edd805cecfef3/coins_detect.jpg)

[크기가 다른 그림에 대해서는 인식이 전혀 안되는 모습](https://kdt-gitlab.elice.io/ai_track/class05/ai_project/team07/7zip/uploads/798c7ab32068e35f2a3a271d451746d9/coins_detect__1_.jpg)

###1.2. HOG필터 + SVM을 이용한 객체 인식 (폐기)

- HOG필터와 SVM은 머신러닝 기반이므로 객체 인식에 적용할 경우, (딥러닝보다) 빠른 연산이 가능하다<br/>→ 보다 많은 Class를 넣을 수 있다

- https://donghwa-kim.github.io/hog.html : HOG필터에 대한 이론

- 간단히 서술하자면 이미지에 Kernel을 적용시키고 Histogram 형태로 Feature Map을 생성

- Hough Transform을 이용한 객체 인식보단 빠르고 인식률이 좋지만, 여전히 많이 정확하지는 않음

- 마찬가지로 이미지의 크기 or 객체의 크기에 영향을 많이 받음

[가우시안 필터로 흐림 효과를 줘도 크기가 다르면 인식이 안됨](https://kdt-gitlab.elice.io/ai_track/class05/ai_project/team07/7zip/uploads/5f420297926004ee85a234345f42fe63/image.png)

### 1.3. YOLO를 이용한 실시간 객체 탐지 딥러닝 모델 이용 (채택)

- 훈련하지 않은 Pre-Train 모델을 사용하여 Box Detection 성능이 좋은 것을 확인

- OneStage 방식이므로 Client의 사용성 측면에서 이점을 가져갈 것으로 예상

#### 1.3.1. 외화 동전의 앞면과 뒷면을 함께 학습 ⟶ ***앞-뒷면을 따로 학습***

- Feature Map을 잘 구별하지 못함

- 동전의 크기에 따라서 구별을 하는 것처럼 보이는데 1.1과 1.2와 같은 문제가 발생함

#### 1.3.2. Gray Scale로 학습 ⟶ ***RGB이미지로 학습***

- 흑백 이미지에서 Feature Map을 잘 구별할 것이라 생각하여 시도

- Recall값이 Spiky한 그래프를 보여주므로 박스를 잘 검출하지 못하는 것으로 보임

- 배경 이미지에 Detection Box를 만드는 등, [객체를 인지하지 못함](https://github.com/KongTi/Project/blob/main/%EC%97%98%EB%A6%AC%EC%8A%A4%20AI%20%EC%84%9C%EB%B9%84%EC%8A%A4%20%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8/grayScaleImage.png?raw=true)

#### 1.3.3. 최신 모델인 YOLOv7 ⟶ ***YOLOv5 X 모델 사용***

- 최신 모델이므로 성능 개선이 있을 것이라 예상<br/>⟶ 오히려 최신 모델이므로 Train되는 Size가 작음 (최적화 등 여러 문제)

- 때문에, 성능 상승폭이 그렇게 크지않고 오히려 하락하는 구간도 발생

- Stable한 YOLOv5에서 가장 큰 X 모델을 채택하여 Train

## 2. Data

- 외화 동전 **데이터는 수가 많지않고** 이미지 Train에는 많은 수의 Train Dataset이 필요함

- 또한, LabelImg Tool로 **직접 라벨링**하여 YOLO Train Set을 구성하는데 많은 시간이 소요됨

- 수집할 데이터의 class는 다음과 같이 선정함

# <한국관광공사 통계 및 통용화폐 거래량을 기반으로 4개국 선정>

## - US (미국) : 통용화폐 거래량 1위 *(25, 10 센트)*
## - JP (일본) : 해외여행 국가 1위, 통용화폐 거래량 3위 *(500, 100 엔)*
## - CN (중국) : 해외여행 국가 2위 *(1 위안)*
## - KR (한국) : 동전 인식 테스트용 *(500, 100 원)*

###2.1. Data Labeling

- Data Imbalance는 직접 데이터를 구글링하여 수집하는 단계에서 **최대한 동일**하게 수집<br/>⟶ 100~110개로 Raw Image가 수집됨

- 일반적으로 LabelImg Tool을 사용하여 직접 Object Box를 만드는 방식<br/>⟶ 시간이 오래 걸리고 효율적이지 못함 (Raw Image 100개)

- CheckingCircle Tool을 제작하여 자동으로 원을 검출하고 Object Box 및 Labeling

### [CheckingCircle](https://github.com/KongTi/CheckingCircle)

- Hough Transform을 이용한 원 객체 검출기
- 검출된 Object Box를 YOLO Dataset의 형태로 저장한다

###2.2. Data Augmentation

- Python의 Albumentations 라이브러리를 이용하여 **데이터 증강**

- Albumentations는 Detection Box도 같이 회전 시켜주므로 적은 데이터를 증강할 때 유리

- 동전은 좌우가 바뀔 일이 없으며, 찍는 사용자에 따라 회전이 될 것이라고 추측

- 때문에 90, 180, 270도로 **이미지를 회전** 시키는 증강 실시<br/>+ 확률적으로 필터 및 노이즈 추가

- KRW 500 Won의 경우, 객체의 크기가 거의 비슷한 데이터가 있음<br/>⟶ **Image Padding**을 넣어 객체의 크기 다양화

### [Augmentation.py](https://github.com/KongTi/CheckingCircle/blob/main/augmentation.py)

### [데이터 증강된 이미지의 수](https://github.com/KongTi/Project/blob/main/%EC%97%98%EB%A6%AC%EC%8A%A4%20AI%20%EC%84%9C%EB%B9%84%EC%8A%A4%20%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8/dataImbalance.jpg?raw=true)

## 3. Train & Test

- [YOLOv5 모델에서 coin.yaml을 설정 해준다](https://github.com/KongTi/YOLOv5/blob/master/data/coin.yaml)

- [YOLOv5 Train 모델의 크기 설정 및 다운로드](https://github.com/ultralytics/yolov5/releases/download/v6.2/yolov5x.pt)

- /models/yolov5_.yaml에서도 class 수에 맞게 nc값 수정

In [ ]:
wandb login # 학습곡선 및 loss 확인용

#리눅스
#sudo wandb login

In [ ]:
python train.py --data ./data/coin.yaml --cfg ./cfg/training/yolov5x.yaml --weights yolov5x.pt --batch-size 8 --epochs 500 --name Train

#리눅스
#sudo nohup python3 train.py --data ./data/coin.yaml --cfg ./models/yolov5x.yaml --weights yolov5x.pt --batch-size 8 --epochs 500 --name Linux_Train

### best.pt ⟶ 가장 좋은 성능 모델로 예측

In [ ]:
python detect.py --source ./data/images/ --weights ./runs/train/Train/weights/best.pt --save-txt

#리눅스
#sudo python3 detect.py --source ./data/images/ --weights ./runs/train/Linux_Train/weights/best.pt --save-txt

### Object Detection의 결과를 Parsing하여 Node.js의 Server에 던져준다

[analysis.py](https://github.com/KongTi/YOLOv5/blob/master/analysis.py)

학습에 따른 Metrics 및 Loss 곡선 [(Wandb)](https://wandb.ai/hjm0137/YOLOv5/runs/2hp6m1bl?workspace=user-)

[Overfitting Loss](https://github.com/KongTi/Project/blob/main/%EC%97%98%EB%A6%AC%EC%8A%A4%20AI%20%EC%84%9C%EB%B9%84%EC%8A%A4%20%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8/overfit.png?raw=true)로 best Epochs은 108로 보임

## 4. Summary

- YOLOv5 X / Epochs 108 / lr 3.17e-3 / SGD / Confidence 0.50

Train

- Precision: 0.96

- Recall: 0.95

- F1 Score: 0.95

Validation, Test

- Precision: 0.96

- Recall: 0.99

- F1 Score: 0.97

[프로젝트 PDF](https://github.com/KongTi/Project/blob/main/%EC%97%98%EB%A6%AC%EC%8A%A4%20AI%20%EC%84%9C%EB%B9%84%EC%8A%A4%20%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8/%EC%97%98%EB%A6%AC%EC%8A%A4%20AI%20%EC%84%9C%EB%B9%84%EC%8A%A4%20%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8.pdf)